# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

Представленный датасет представляет из себя имформацию о количестве звонков, их длительности, количестве трафика в мегабайтах и количестве отпраленных СМС за один месяц. Так же есть информация о тарифе пользователя. Описание данных:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).


Для решения поставленной задачи необходимо импортировать следующие библиотеки

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings("ignore")

## 1. Откройте и изучите файл

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Как можно увидеть, датасет содержит данные о 3214 пользователях, пропусков в данных нет, так как эти данные были предобработаны. Пример одной записи представлен ниже

In [4]:
df.sample()

,calls,minutes,messages,mb_used,is_ultra
2487,67.0,439.01,20.0,24095.57,0


In [5]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

#### Выводы
- Данные содержат в себе информацию о предоставленных услугах для 3214 пользователей
- Известен тариф пользователя
- Данных по тарифу "Смарт" больше, чем по тарифу "Ультра"

## 2. Разбейте данные на выборки

Для начала нужно разделить набор данных на признаки и метки классов

In [6]:
X = df.drop('is_ultra', axis=1)
y = df['is_ultra'] 

После чего необходимо "разрезать" данные на обучающую и валидационно-тестовую выборку. Для этого используется функция train_test_split. Соотношение обучающей части к проверочной выбрано 80:20. Так же важным является параметр stratify для сохранения балансов классов при нарезке данных 

In [7]:
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.2, random_state=12345, stratify=y)

После чего валидационно-тестовую выборку необходимо разделить пополам для получения валидационной и тестовой выборки :)

In [8]:
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=12345, stratify=y_val_test)

Необходимо проверить, корректно ли отработал данный параметр, для этого посчитаем соотношение классов в исходном наборе данных: <b>2229/985 = 2.26</b> и в обучающей выборке <b>1783/788 = 2.26</b>. Отлично! 

In [9]:
y_train.value_counts()

0    1783
1     788
Name: is_ultra, dtype: int64

In [10]:
X_train.shape

(2571, 4)

Посчитаем соотношение классов в валидационной выборке: <b>223/98 = 2.28</b>. Вполне сопоставимые значения

In [11]:
y_val.value_counts()

0    223
1     98
Name: is_ultra, dtype: int64

Как видно ниже, соотношение классов примерно такое же

In [12]:
y_test.value_counts()

0    223
1     99
Name: is_ultra, dtype: int64

#### Выводы
- Получены обучающая, валидационная и тестовая выборки с соотношением балансов классов
- Соотношение баланса было проверено

## 3. Исследуйте модели

Изначально я попытался подобрать оптимальные параметры для алгоритмов путем перебора, но для дерева решений и случайного леса я смог реализовать такой механизм, то для логической регрессии так подбирать параметры было несколько неудобно

In [13]:
for depth in range(2, 19):
    model = DecisionTreeClassifier(random_state=123, max_depth=depth)
    model.fit(X_train, y_train)
    predictions_valid = model.predict(X_val)
    train_score = model.score(X_train, y_train)
    score = model.score(X_val, y_val)
    print('Глубина обучения для дерева:{}'.format(depth))
    print('Точность классификации на обучающей выборке:{}'.format(train_score))
    print('Точность классификации на валидационной выборке:{}'.format(score))
    print('________________________________________________________________________')

Глубина обучения для дерева:2
Точность классификации на обучающей выборке:0.7810190587320109
Точность классификации на валидационной выборке:0.7538940809968847
________________________________________________________________________
Глубина обучения для дерева:3
Точность классификации на обучающей выборке:0.7969661610268378
Точность классификации на валидационной выборке:0.7725856697819314
________________________________________________________________________
Глубина обучения для дерева:4
Точность классификации на обучающей выборке:0.8063010501750292
Точность классификации на валидационной выборке:0.7912772585669782
________________________________________________________________________
Глубина обучения для дерева:5
Точность классификации на обучающей выборке:0.8199144301828083
Точность классификации на валидационной выборке:0.7881619937694704
________________________________________________________________________
Глубина обучения для дерева:6
Точность классификации на обучающей вы

In [14]:
for n_est in range(2, 10, 2):
    for deep in range(2,11):
        model = RandomForestClassifier(random_state=123, n_estimators=n_est, max_depth=deep)
        model.fit(X_train, y_train)
        predictions_valid = model.predict(X_val)
        train_score = model.score(X_train, y_train)
        score = model.score(X_val, y_val)
        print('Количество деревьев:{}'.format(n_est))
        print('Глубина обучения для дерева:{}'.format(deep))
        print('Точность классификации на обучающей выборке:{}'.format(train_score))
        print('Точность классификации на валидационной выборке:{}'.format(score))
        print('________________________________________________________________________')

Количество деревьев:2
Глубина обучения для дерева:2
Точность классификации на обучающей выборке:0.7926876701672501
Точность классификации на валидационной выборке:0.8193146417445483
________________________________________________________________________
Количество деревьев:2
Глубина обучения для дерева:3
Точность классификации на обучающей выборке:0.793465577596266
Точность классификации на валидационной выборке:0.8193146417445483
________________________________________________________________________
Количество деревьев:2
Глубина обучения для дерева:4
Точность классификации на обучающей выборке:0.7992998833138857
Точность классификации на валидационной выборке:0.8161993769470405
________________________________________________________________________
Количество деревьев:2
Глубина обучения для дерева:5
Точность классификации на обучающей выборке:0.8129132633216647
Точность классификации на валидационной выборке:0.794392523364486
_______________________________________________________

Но я узнал о методе поиска оптимальных параметров GridSearchCV. после чего было решено искать оптимальные параметры с помощью данного метода. Но так как с каждым запуском значения менялись (так как внутри этого метода происходит постоянное обучение) было необходимо запустить этот поиск несколько раз и посмотреть, какие примерно значения он будет возвращать. Оценка точности производится с помощью валидационной выборки.

In [15]:
param_grid={'max_depth': [deep for deep in range (2, 20)]}
gs_dt = GridSearchCV(DecisionTreeClassifier(), param_grid=param_grid)
gs_dt.fit(X_train, y_train)
gs_dt.best_params_

{'max_depth': 7}

Пять запусков поиска наилучшего параметра для дерева решений вернуло значение 7, его и возьму

In [16]:
param_grid={'max_depth': [deep for deep in range (2, 10)],
           'n_estimators': [n_est for n_est in range (10, 100, 10)]}
gs_rf = GridSearchCV(RandomForestClassifier(), param_grid=param_grid)
gs_rf.fit(X_train, y_train)
gs_rf.best_params_

{'max_depth': 9, 'n_estimators': 40}

Поиск оптимальных параметров для случайного леса достаточно тяжеловесная  задача, поэтому было решено ограничить глубину каждого дерева в промежутке от 2 до 10 и количество деревьев от 10 до 100 с шагом в 10 деревьев

Первый запуск вернул следующие значения:
- Глубина дерева - 9
- 30 деревьев

Второй запуск вернул следующие значения:
- Глубина дерева - 8
- 50 деревьев

Третий запуск вернул следующие значения:
- Глубина дерева - 9
- 90 деревьев

Четвертый запуск вернул следующие значения:
- Глубина дерева - 9
- 60 деревьев

Пятый запуск вернул следующие значения:
- Глубина дерева - 9
- 60 деревьев

С глубиной дерева было определиться достаточно просто, а вот с количеством все не так однозначно. Было решено выбрать 60 деревьев, так как это единственное повторяющееся значение, плюс лежит примерно в середине разброса.

In [17]:
param_grid={'penalty': ['l1', 'l2'],
           'C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}
gs_lr = GridSearchCV(LogisticRegression(), param_grid=param_grid)
gs_lr.fit(X_train, y_train)
gs_lr.best_params_

{'C': 0.4, 'penalty': 'l1'}

Для логистической регрессии изучалось влияние двух параметров: penalty и C. Для параметра penalty использовались два возможных значения: l1 и l2, параметр C варьировался от 0.1 до 1.0 с шагом 0.1

Первый запуск вернул следующие значения:
- l1
- С = 1

Второй запуск вернул следующие значения:
- l1
- С = 0.7

Третий запуск вернул следующие значения:
- l1
- С = 0.7

Четвертый запуск вернул следующие значения:
- l1
- С = 0.6

Пятый запуск вернул следующие значения:
- l1
- С = 0.7

Итого были выбраны параметры penalty = l1 и С = 0.7

#### Выводы
- Параметры для дерева решений: глубина = 7
- Параметры для случайного леса: глубина = 9 и количество деревьев = 60
- Параметры для логистической регрессии: penalty = l1 и С = 0.7

## 4. Проверьте модель на тестовой выборке

Обучим дерево решений с параметрами, полученными в предыдущем пункте и проверим точность классификации на тестовом наборе данных

In [18]:
model = DecisionTreeClassifier(random_state=123, max_depth=7)
model.fit(X_train, y_train)
predictions_test = model.predict(X_test)
train_score = model.score(X_train, y_train)
score = model.score(X_test, y_test)
print('Точность классификации на обучающей выборке:{}'.format(train_score))
print('Точность классификации на тестовой выборке:{}'.format(score))

Точность классификации на обучающей выборке:0.8506417736289381
Точность классификации на тестовой выборке:0.7981366459627329


Сделаем тоже самое для случайного леса

In [19]:
model = RandomForestClassifier(random_state=123, n_estimators=60, max_depth=9)
model.fit(X_train, y_train)
predictions_valid = model.predict(X_val)
train_score = model.score(X_train, y_train)
score = model.score(X_test, y_test)
print('Точность классификации на обучающей выборке:{}'.format(train_score))
print('Точность классификации на тестовой выборке:{}'.format(score))

Точность классификации на обучающей выборке:0.882535978218592
Точность классификации на тестовой выборке:0.8012422360248447


И для логистической регрессии

In [20]:
model = LogisticRegression(random_state=123, C=0.7, penalty='l1')
model.fit(X_train, y_train)
predictions_valid = model.predict(X_val)
train_score = model.score(X_train, y_train)
score = model.score(X_test, y_test)
print('Точность классификации на обучающей выборке:{}'.format(train_score))
print('Точность классификации на тестовой выборке:{}'.format(score))

Точность классификации на обучающей выборке:0.7479579929988331
Точность классификации на тестовой выборке:0.7360248447204969


#### Выводы
Среди выбранных моделей лишь две отвечают необходимому порогу точности: дерево решений с точностью классификации на тесте 0.798 и случайный лес с точностью классификации на тесте 0.801. Логистическая регрессия, к моему удивлению, с поставленной задачей не справилась, точность классификации на тестовом наборе достигла лишь 0.736 

## 5. (бонус) Проверьте модели на адекватность

Для начала надо посомтреть, какой класс является доминирующим в исходном наборе данных

In [21]:
y_train.value_counts()

0    1783
1     788
Name: is_ultra, dtype: int64

Таким классом оказался класс с меткой 0, он встречается 1783 раза. Также необходимо посомтреть, какой объем данных нам предоставлен

In [22]:
y_train.count()

2571

Всего у нас 2571 запись. Соответственно, если мы проставим всем данным метку класса, равную 0, то получим точность классификации равную 0.7. Из этого следует, если модель имеет точность классификации на тестовом наборе ниже 0.7, то данная модель неадекватна, а если больше - то адекватна

In [23]:
1783 / 2571

0.6935044729677169

#### Выводы
Так как выбранная модель дает точность в 0.8, то она предсказывает лучше, чем если бы мы на проверке проставили всем объектам класс, равный наибольшему в обучающей выборке (метка класса 0). в таком случае точность классификации была бы равна 0.7, что меньше, чем точность полученной модели. Из этого следует что модель адекватна

# Общие выводы
- Была произведена разбивка исходных данных на обучающую, валидационную и тестовую выборки в соотношении 80-10-10 соответственно, при этом в этих подвыборках остался изначальный баланс классов
- Для обучения было выбрано три модели: дерево решений, случайный лес и логистическая регрессия (не стал брать больше моделей, так как в тренажере были рассмотрены только они)
- Были подобраны оптимальные параметры для моделей методом GridSearchCV для каждой модели
- Проведено тестирование точностей моделей, дерево решений показало точность, равную 0.798, случайный лес оказался немного точнее - 0.801, а вот логистическая регрессия подвела, показав точность, равную 0.736, которой недостаточно для прохождения задания
- Все три модели прошли проверку на адекватность, показав значение лучше, чем на константной модели

<b>Лучше всех себя показал случайный лес с параметрами n_estimators=60, max_depth=9, показав точность на тестовой выборке в 0.801</b>